In [ ]:
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from arch import arch_model

import warnings
warnings.filterwarnings("ignore")

In [ ]:
# Fontsizes
LEGEND_SIZE = 12
AXIS_LABEL_SIZE = 14
TICK_LABEL_SIZE = 12
TITLE_SIZE = 16

# Line width
LINEWIDTH = 1.2

START_DATE = '2020-01-01'
END_DATE = '2021-01-01'

data = yf.download('BZ=F', start=START_DATE, end=END_DATE, progress=False)[['Adj Close']]

# Log returns
data['Log Returns'] = np.log(data['Adj Close']).diff()

# Annualized rolling standard deviation
window_size = 21
data['Rolling std'] = data['Log Returns'].rolling(window_size).std() * np.sqrt(252)

# Historical standard deviation
data['Historical std'] = data['Log Returns'].std() * np.sqrt(252)

# create GARCH(1,1) model
model = arch_model(data['Log Returns'].dropna(), vol='Garch', p=1, q=1)
model_fit = model.fit(disp='off')
data['GARCH'] = model_fit.conditional_volatility * np.sqrt(252)

fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 8), sharex=True, dpi=150)

ax3 = ax1.twinx()

ax1.set_title('NYMEX 2023 Brent Crude Oil futures prices and volatility', fontsize=TITLE_SIZE)

ax1.plot(
    data['Adj Close'], 
    lw=LINEWIDTH, 
    color='#2a694c', 
    label='Brent Crude Oil Last Day Futures'
)
ax1.grid(alpha=0.2)
ax1.set_axisbelow(True)
ax1.set_ylabel('Price USD', fontsize=AXIS_LABEL_SIZE)
ax1.set_ylim(ax1.get_ylim()[0] * 0.8, ax1.get_ylim()[1] * 1.2)

ax3.plot(
    data['Log Returns'], 
    lw=LINEWIDTH, 
    label='Log Returns', 
    color='tab:gray', 
    alpha=0.4
)
ax3.set_ylabel('Log Returns', fontsize=AXIS_LABEL_SIZE)
ax3.set_ylim(ax3.get_ylim()[0] * 1.2, ax3.get_ylim()[1] * 1.2)

# combine ax1 and ax3 legend
lines, labels = ax1.get_legend_handles_labels()
lines2, labels2 = ax3.get_legend_handles_labels()
ax3.legend(lines + lines2, labels + labels2, frameon=False, loc='upper left', fontsize=LEGEND_SIZE)

ax2.plot(
    data['Historical std'], 
    lw=LINEWIDTH, 
    label='Historical Standard Deviation', 
    color='#e39520'
)
ax2.plot(
    data['Rolling std'], 
    lw=LINEWIDTH, 
    label=f'Rolling {window_size}-day Standard Deviation', 
    color='#801b20'
)
ax2.plot(
    data['GARCH'], 
    lw=LINEWIDTH, 
    label='GARCH(1,1) Model', 
    color='#29a5cf'
)
ax2.legend(frameon=False, loc='upper left', fontsize=LEGEND_SIZE)
ax2.grid(alpha=0.2)
ax2.set_axisbelow(True)
ax2.set_xlabel('Date', fontsize=AXIS_LABEL_SIZE)
ax2.set_ylabel('Annualized Volatility', fontsize=AXIS_LABEL_SIZE)
ax2.set_ylim(0.1, ax2.get_ylim()[1] * 1.2)

# make the label marker bigger
fig.tight_layout(pad=0.1, h_pad=-0.35)

for ax in fig.axes:
    ax.tick_params(axis='both', which='major', labelsize=TICK_LABEL_SIZE)

fig.savefig('volatility_demo.png', dpi=200, bbox_inches='tight')